## Robust/Sparse Logistic Regression MIO Formulation

In [2]:
using CSV, DataFrames, Gurobi, JuMP

In [3]:
df = CSV.File("Mappe1.csv", delim=";") |> DataFrame
dropmissing!(df)
X = Matrix(df[:, 1:end-1])
y = df[:, end];

In [ ]:
#Define indices
n, p = size(X)
rho = 1
gamma = 5
M = 100
k = 5

# Initialize model
logistic_model = Model(Gurobi.Optimizer)

# Add variables
@variable(logistic_model, beta[j=1:p])
@variable(logistic_model, 0<=z[j=1:p]<=1, Bin)
@variable(logistic_model, beta0)
@variable(logistic_model, mu <= 0)
@variable(logistic_model, nu[i=1:n] <= 0)

# Add constraints
@constraint(logistic_model, constraint1[i in 1:n], mu + nu[i] <= log((1+exp(-y[i]*(sum(X[i,j]*beta[j] for j=1:p)+beta0))
            +rho*sqrt(sum((beta[j])^2 for j=1:p)))/(1+exp(y[i]*(sum(X[i,j]*beta[j] for j=1:p)+beta0))
            +rho*sqrt(sum((beta[j])^2 for j=1:p)))))
@objective(logistic_model, Max, -sum(log(1+exp(-y[i]*sum(X[i,j]*beta[j] for j=1:p)
            +rho*sqrt(sum((beta[j])^2 for j=1:p)))) for i=1:n)+gamma*mu+sum(nu[i] for i=1:n))
@constraint(logistic_model, bigM1[j in 1:p], -M*z[j] <= beta[j])
@constraint(logistic_model, bigM2[j in 1:p], beta[j] <= M*z[j])
@constraint(logistic_model, sparsity, sum(z[j] for j=1:p) <= k)

optimize!(logistic_model)

In [12]:
using CSV, DataFrames, Gurobi, JuMP

df = CSV.File("Mappe1.csv", delim=";") |> DataFrame
dropmissing!(df)
X = Matrix(df[:, 1:end-1])
y = df[:, end];

# Define indices
n, p = size(X)
rho = 1
gamma = 5
M = 100
k = 5

# Initialize model
logistic_model = Model(Gurobi.Optimizer)

# Add variables
@variable(logistic_model, beta[j=1:p])
@variable(logistic_model, 0 <= z[j=1:p] <= 1, Bin)
@variable(logistic_model, beta0)
@variable(logistic_model, mu <= 0)
@variable(logistic_model, nu[i=1:n] <= 0)

# Hinge loss approximation
@variable(logistic_model, u[i=1:n, j=1:p] >= 0)
@variable(logistic_model, v[i=1:n, j=1:p] >= 0)

# Add constraints
@constraint(logistic_model, constraint1[i in 1:n, j=1:p], u[i, j] - v[i, j] == y[i] * (sum(X[i, k] * beta[k] for k = 1:p) + beta0) - 1 + nu[i])
@constraint(logistic_model, bigM1[i=1:n, j=1:p], u[i, j] <= M * (1 - z[j]))
@constraint(logistic_model, bigM2[i=1:n, j=1:p], v[i, j] <= M * z[j])
@constraint(logistic_model, sparsity, sum(z[j] for j = 1:p) <= k)

# Objective using hinge loss
@objective(logistic_model, Max, -sum(u[i, j] + v[i, j] for i in 1:n, j in 1:p) / n + gamma * mu + sum(nu[i] for i in 1:n))

optimize!(logistic_model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 147187 rows, 101926 columns and 412347 nonzeros
Model fingerprint: 0xd09fa57f
Variable types: 101913 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e-04, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.08 work units)
Thread count was 1 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -

User-callback calls 45, time in user-callback 0.00 sec
